In [ ]:
import os

os.chdir(r'/Users/jialunlyu/Documents/GitHub/ProjectBuffalo')

In [ ]:
import pandas as pd
import numpy as np
import buffalo.algorithm.outliers as ol_algo 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from statsmodels.tsa.api import ArmaProcess

In [ ]:
armaprocess = ArmaProcess(ar=np.array([0.75]), ma=np.array([0.23]))

In [ ]:
obs = armaprocess.generate_sample(5000)

In [ ]:
plt.plot(obs)

In [ ]:
def add_outliers(obs, ol_info):
    obs = obs.copy()
    added = []
    for i in range(len(ol_info.index)):
        position = np.random.choice(range(len(obs)), ol_info.iloc[i]['Num'])
        effect = np.random.normal(ol_info.iloc[i]['effect_mean'], ol_info.iloc[i]['effect_std'], ol_info.iloc[i]['Num'])
        if ol_info.iloc[i]['type'] == 'AO':
            obs[position] += effect
        added.append(pd.merge(
            pd.DataFrame({'position': position, 'effect': effect}),
            ol_info,
            how='cross'
        ))
    return obs, pd.concat(added, axis=0)

In [ ]:
ol_type = pd.DataFrame({
    'type': ['AO'],
    'Num': [100],
    'effect_mean': [20],
    'effect_std': [4]
})

In [ ]:
ol_obs, ol_info = add_outliers(obs.copy(), ol_type)

In [ ]:
plt.plot(ol_obs)

In [ ]:
ol_detection = ol_algo.IterativeTtestOutlierDetection(
    types= pd.DataFrame({'type': ['AO']}),
    maxit=2,
    maxit_iloop=4,
    maxit_oloop=4,
    cval=None,
    cval_reduce=0.14286,
    discard_method='en-masse',
    discard_cval=None,
    tsmethod='ARIMA',
    args_tsmethod={
        "order": (1,0,1),
        "seasonal_order": (0, 0, 0, 0),
        "with_intercept": False
    }
)

In [ ]:
result, endog, exog = ol_detection.fit(endog=pd.DataFrame(ol_obs)[0], exog=None)

In [ ]:
plt.plot(endog)